In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/f/Documents/Ironhack/data-wrangling-project/charging_data.csv")

In [ ]:
df

In [ ]:
df["art_der_ladeeinrichung"].value_counts()
#translate the language with mapping function

In [ ]:
df["kreis_kreisfreie_stadt"].value_counts()

In [ ]:
df["anzahl_ladepunkte"].value_counts()

In [ ]:
df2 = pd.read_csv('/Users/f/Documents/Ironhack/data-wrangling-project/46241-0011_$F.csv')

In [ ]:
import requests
response = requests.get("https://ev-database.org/#sort:path~type~order=.rank~number~desc|rs-price:prev~next=10000~100000|rs-range:prev~next=0~1000|rs-fastcharge:prev~next=0~1500|rs-acceleration:prev~next=2~23|rs-topspeed:prev~next=110~350|rs-battery:prev~next=10~200|rs-towweight:prev~next=0~2500|rs-eff:prev~next=100~300|rs-safety:prev~next=-1~5|paging:currentPage=0|paging:number=9",headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'})
response

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
car_list = soup.find_all("href", class_= "title")
car_list

In [ ]:
registration = pd.read_csv("/Users/f/Documents/Ironhack/data-wrangling-project/Electric_Vehicle_Population_Data.csv")

In [ ]:
registration

In [ ]:
price = pd.read_csv("/Users/f/Documents/Ironhack/data-wrangling-project/car_prices.csv")

In [ ]:
price

In [ ]:
price["make"].value_counts().head(20)

In [ ]:
import requests
from bs4 import BeautifulSoup

# Main URL to start with
main_url = "https://www.fueleconomy.gov/feg/"
response = requests.get(main_url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the URL for the specific page containing emission data
extend_url = "https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=45751"
response_2 = requests.get(extend_url)
soup2 = BeautifulSoup(response_2.content, "html.parser")

emision_data = []

# Extract emission data from the second page
emission_data = soup2.find_all("img", src = "/feg/photos/2023_Chevy_Bolt_EV.jpg")
emission_data.append(emission_data)

# Print or process the extracted emission data
print(emission_data)

In [ ]:
main_url = [i.get("https://www.fueleconomy.gov/feg/") for i in soup.find_all('img', src ='/feg/photos/2023_Chevy_Bolt_EV.jpg')]

emission_data = []

for href in href_list:
    response = requests.get('https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=45751' + src, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract actors from the cast section
    emission_data = [emission_data.get_text() for emission_data in soup.find_all('a', attrs={'data-testid': 'title-cast-item__actor'})]
    
    # Append the list of actors to the all_actors list
    emission_data.append(emission_data)


In [3]:
import requests
from bs4 import BeautifulSoup

# Main URL
main_url = "https://www.fueleconomy.gov/feg/"
response = requests.get(main_url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the URLs of the pages containing emission data
extend_url = "https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=45751"
response_2 = requests.get(extend_url)
soup2 = BeautifulSoup(response_2.content, "html.parser")

# List to store emission data
emission_data = []

# Extract emission data from the page (replace with relevant data extraction code)
# For example:
emission_data.append(soup2.find_all("0", class_ = "sbsCellData"))

# Print or process the extracted emission data
print(emission_data)


[[]]


In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the page containing the emission data
url = "https://www.fueleconomy.gov/feg/Find.do?action=sbs&id=45751"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the emission data
emission_data = soup.find_all("td", class_="ghg-score")

print(emission_data)

[]


In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Main URL
main_url = "https://www.fueleconomy.gov/feg/"
response = requests.get(main_url)
soup = BeautifulSoup(response.content, "html.parser")

url = "https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2019&year2=2024&cbmkChevrolet=Chevrolet&cbmkFord=Ford&cbmkNissan=Nissan&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&YearSel=2019-2024&make=Chevrolet%3B+Ford%3B+Nissan&mclass=&vfuel=&vtype=&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=1"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

emissions = []

# Find all emission components
components = soup.find_all("div", class_="ghg-score")

for component in components:
    # Extract emission text
    emission_text = component.get_text
    # Append to emissions list
    emissions.append(emission_text)

# Create DataFrame
df = pd.DataFrame({'emission': emissions})
emissions

#Display the DataFrame
#emissions = float[(x.split()[0]) for x in emissions if 'grams' in x]
#emission_value = emissions[0] if emissions else None
#emission_value

emissions = [x.split()[0] for x in emissions if 'grams' in x]
df = pd.DataFrame({'emissions': emissions})
print(df)

# Extract the emission value without brackets
emission_value = (emissions[0]) if emissions else None
emission_value

TypeError: argument of type 'method' is not iterable

In [12]:
# Display the DataFrame
emissions = [x.split()[0] for x in emissions if 'grams' in x]
df = pd.DataFrame({'emissions': emissions})
print(df)

# Extract the emission value without brackets
emission_value = float(emissions[0]) if emissions else None
print(emission_value)

TypeError: argument of type 'method' is not iterable

SyntaxError: invalid syntax (3584659475.py, line 2)

In [5]:
# Extract only the numerical value without the unit from the "emission" column
df['emission'] = df['emission'].apply(lambda x: x.split()[0] if len(x.split()) > 0 else None)

# Filter the DataFrame to include only rows where the emission value is "0"
filtered_df = df[df['emission'] == '0']

# Print the filtered DataFrame
print(filtered_df)


   emission
16        0


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_emissions(manufacturer):
    # Construct the URL based on the manufacturer
    url = f"https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2019&year2=2024&cbmk{manufacturer}={manufacturer}&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&YearSel=2019-2024&MakeSel={manufacturer}&MarClassSel=&FuelTypeSel=&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=10"

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find emission data in the HTML
    emission_data = soup.find("td", class_="sbsCellData").get_text(strip=True)

    # Return the emission data
    return emission_data

# List of manufacturers to scrape data for
manufacturers = ['Chevrolet', 'Ford', 'Nissan']

# Iterate over the manufacturers and scrape emissions data
for manufacturer in manufacturers:
    emission_data = scrape_emissions(manufacturer)
    print(f"Emissions data for {manufacturer}: {emission_data}")

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_emissions(manufacturer):
    # Construct the URL based on the manufacturer
    url = f"https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2019&year2=2024&cbmk{manufacturer}={manufacturer}&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&YearSel=2019-2024&MakeSel={manufacturer}&MarClassSel=&FuelTypeSel=&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=10"

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find emission data in the HTML
    emission_element = soup.find("td", class_="sbsCellData")
    
    # Check if emission data is found
    if emission_element:
        # Extract text if element is found
        emission_data = emission_element.get_text(strip=True)
    else:
        # If element is not found, return None
        emission_data = None

    # Return the emission data
    return emission_data

# List of manufacturers to scrape data for
manufacturers = ['Chevrolet', 'Ford', 'Nissan']

# Iterate over the manufacturers and scrape emissions data
for manufacturer in manufacturers:
    emission_data = scrape_emissions(manufacturer)
    print(f"Emissions data for {manufacturer}: {emission_data}")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrape_emissions(manufacturer, model):
    # Construct the URL based on the manufacturer and model
    url = f"https://www.fueleconomy.gov/feg/PowerSearch.do?action=PowerSearch&year1=2019&year2=2024&cbmk{manufacturer}={model}&minmsrpsel=0&maxmsrpsel=0&city=0&highway=0&combined=0&YearSel=2019-2024&MakeSel={manufacturer}&MarClassSel=&FuelTypeSel=&VehTypeSel=&TranySel=&DriveTypeSel=&CylindersSel=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=10"

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find emission data in the HTML
    emission_element = soup.find("td", class_="sbsCellData")
    
    # Check if emission data is found
    if emission_element:
        # Extract text if element is found
        emission_data = emission_element.get_text(strip=True)
    else:
        # If element is not found, return None
        emission_data = None

    # Return the emission data
    return emission_data

# List of Chevrolet models
chevrolet_models = ['Bolt', 'Camaro', 'Equinox', 'Malibu', 'Silverado', 'Suburban', 'Tahoe', 'Traverse', 'Trax']

# Scrape emissions data for each Chevrolet model
for model in chevrolet_models:
    manufacturer = 'Chevrolet'
    emission_data = scrape_emissions(manufacturer, model)
    print(f"Emissions data for {manufacturer} {model}: {emission_data}")


In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


# Task: Columns -> Car Type, Emission and Price

In [ ]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    for m in maker:
        page = 1
        while True:
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=50"
            response = r.get(maker_url)
            soup = bs(response.content)
        except:
            
        link = [x.get("href") for x in soup.select("a[href]")[154:210] if "yourMpgVehicle" not in x.get("href")]

        for l in link:
            response = r.get(base+l)
            soup = bs(response.content)
            type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
            
            if type == "Electricity":
                battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                gas_tank.append(np.nan)
                mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                mpg.append(np.nan)
            elif type in ["Regular Gasoline", "Diesel"]:
                battery.append(np.nan)
                gas_tank.append(soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0]) #gallons
                mpge.append(np.nan)
                mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
            else:
                battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                gas_tank.append(soup.find_all("td", class_="sbsEconData")[3].getText().split(" ")[0]) #gallons
                mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])

    print("Creating Dictionary.")
    manufacturer_dict = {"electric_consumption":mpge, "electric_storage":battery,
                        "analogue_consumption":mpg, "analogue_storage":gas_tank}
    print("Returning Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df = get_data(base_url, makers)
print("Finished")

In [ ]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

In [ ]:
def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    for m in maker:
        page = 1
        while True:
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=50"
            response = r.get(maker_url)
            soup = bs(response.content)
        except:
            
        link = [x.get("href") for x in soup.select("a[href]")[154:210] if "yourMpgVehicle" not in x.get("href")]

        for l in link:
            response = r.get(base+l)
            soup = bs(response.content)
            type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
            
            if type == "Electricity":
                battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                gas_tank.append(np.nan)
                mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                mpg.append(np.nan)

print("Creating Dictionary.")
    manufacturer_dict = {"electric_consumption":mpge, "electric_storage":battery,
                        "analogue_consumption":mpg, "analogue_storage":gas_tank}
    print("Returning Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df = get_data(base_url, makers)
print("Finished")